<a href="https://colab.research.google.com/github/Manpreetkour95/NLP/blob/main/Faiss_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 15.7 MB/s eta 0:00:00


In [ ]:
 pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.4 MB/s eta 0:00:00


In [ ]:
import faiss
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.cluster import KMeans
import numpy as np

# Step 1: Install required libraries (if not already installed)
# pip install faiss-cpu
# pip install transformers

# Step 2: Import the necessary libraries

# Step 3: Load DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)

# Step 4: Prepare your text data and obtain embeddings
texts = ["Your", "list", "of", "text", "sentences", "to", "cluster"]

# Tokenize and obtain embeddings
input_ids = []
for text in texts:
    input_ids.append(tokenizer.encode(text, add_special_tokens=True))
inputs = torch.LongTensor(input_ids)
with torch.no_grad():
    embeddings = model(inputs).last_hidden_state.mean(dim=1).numpy()

# Step 5: Perform clustering using Faiss

# Define the number of clusters
num_clusters = 3

# Initialize a K-Means clustering model
kmeans = KMeans(n_clusters=num_clusters)

# Fit the K-Means model on the embeddings
kmeans.fit(embeddings)

# Get cluster assignments for each sentence
cluster_assignments = kmeans.labels_

# Step 6: Explore the cluster assignments

# Now, you can explore the cluster assignments and group your sentences based on their clusters.
# cluster_assignments contains the cluster number to which each sentence belongs.

for cluster_id in range(num_clusters):
    cluster_sentences = [texts[i] for i, label in enumerate(cluster_assignments) if label == cluster_id]
    print(f"Cluster {cluster_id + 1}:")
    for sentence in cluster_sentences:
        print(f"- {sentence}")


Cluster 1:
- list
- text
Cluster 2:
- Your
- of
- to
Cluster 3:
- sentences
- cluster


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd

data = {
    'names': ['John Doe', 'Jane Smith', 'David Johnson', 'Sarah Williams', 'Michael Brown'],
    'Age': [30, 25, 35, 28, 32],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Miami']
}

df = pd.DataFrame(data)
import re
import difflib
import pandas as pd

l_check = ['llc', 'inc', 'ltd', 'co', 'llp', 'limited', 'lp', 'corp', 'corporation']

class NameMatcher:
    def __init__(self, name_list):
        self.name_list = name_list

    @staticmethod
    def preprocessing(text):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
        if '&' in text.split():
            text = re.sub(r'[&]', 'and', text)  # Replace '&' with 'and'
        if len(text.lower().split()) > 1 and text.lower().split()[-1] in l_check:
            text = " ".join(text.split()[0:-1])  # Remove common suffixes
        return text

    def name_similarity(self, text1, text2):
        tokens1 = text1.lower().split()
        tokens2 = text2.lower().split()

        intersection = set(tokens1) & set(tokens2)
        union = set(tokens1) | set(tokens2)
        jaccard_similarity = len(intersection) / len(union)

        sequence_similarity = difflib.SequenceMatcher(None, tokens1, tokens2).ratio()

        confidence = 0.6 * jaccard_similarity + 0.4 * sequence_similarity  # Weighted average
        return confidence

    def find_closest_matches(self, input_name, num_matches=1):
        distances = [(name, self.name_similarity(input_name, name)) for name in self.name_list]
        sorted_distances = sorted(distances, key=lambda x: x[1], reverse=True)
        closest_matches = sorted_distances[:num_matches]

        return closest_matches

    def all_matches(self, input_name_list):
        match_results = []
        for input_name in input_name_list:
            input_name = self.preprocessing(input_name)
            match_results.append(self.find_closest_matches(input_name))

        return match_results

    def best_matched_name(self, input_name_list):
        sorted_matches = [matches[0][0] if matches else None for matches in self.all_matches(input_name_list)]
        return sorted_matches

# Load data from a CSV file
df['names'] = df['names'].apply(NameMatcher.preprocessing)  # Apply preprocessing to the 'names' column
name_list = df['names'].tolist()

# Create an instance of the NameMatcher class
name_matcher = NameMatcher(name_list)

# Example usage:
input_names = ["input_name1", "input_name2", "input_name3"]
best_matches = name_matcher.best_matched_name(input_names)
print(best_matches)
